# Cut maps into smaller training items

Vertex AutoML image object detection has a limit of 500 for labeled boxes in a training image. Often times, fantasy maps have more than 24 x 24 (square root of 500, rounded down) cells present.

To ensure higher recall for trained models, we will take larger images and cut them into smaller images, or "shards." Each image will be reduced into a set of 24 x 24 or smaller images. Smaller images will be calculated from the top-left corner and separating at 24, with remainders kept as smaller shards. There will be a limited amount of overlap between sharded images--no more than 1 cell.

In [101]:
%%writefile requirements.txt

google-cloud-aiplatform
google-cloud-firestore
google-cloud-storage
Pillow
imgaug
imageio

Overwriting requirements.txt


In [2]:
!pip install -r requirements.txt

In [3]:
# Get your GCP project id from gcloud
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
PROJECT_ID=shell_output[0]
print("Project ID: ", PROJECT_ID)

Project ID:  fantasymaps-334622


In [4]:
BUCKET="fantasy-maps"

## Query Firestore for map data

In [5]:
from google.cloud import firestore

client = firestore.Client()
collection = client.collection("FantasyMaps")
query = collection.where("vtt", "!=", "").limit(5).get()

## Download images from GCS

In [6]:
from google.cloud import storage

storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(BUCKET)

In [7]:
metadata = []
for _, v in enumerate(query):
    ddict = v.to_dict()
    gcs = ddict["gcsURI"]
    path =  gcs.split("/")
    rel_path = "/".join(path[3:])
    print(rel_path)
    blob = bucket.get_blob(rel_path)
    fname = f"sample_data/{path[-1]}"
    blob.download_to_filename(fname)
    ddict["local"] = fname
    metadata.append(ddict)

ScrapedData/15x60_illmarsh_tunnels_under_t.15x60.jpg
ScrapedData/here's_a_little_mine_for_you._.40x40.jpg
ScrapedData/ruined_auditorium_[40x40].40x40.jpg
ScrapedData/a_big_dungeon_for_the_next_tim.80x80.jpg
ScrapedData/the_old_tower_near_the_village.20x20.jpg


## Compute shards


In [8]:
def create_image_shard_coords(vtt):
    import math
    DIM = 24
    shard_top_left = []
    
    c_shards = math.floor(vtt["cols"] / DIM)
    r_shards = math.floor(vtt["rows"] / DIM)
    
    if c_shards == 0:
        shard_width = vtt["imageWidth"]
    else:
        shard_width = DIM * vtt["cellWidth"]
        
    if r_shards == 0:
        shard_height = vtt["imageHeight"]
    else:
        shard_height = DIM * vtt["cellHeight"]
    
    curr_width = 0
    curr_height = 0
    
    while curr_width <= vtt["imageWidth"]:
        while curr_height <= vtt["imageHeight"]:
            shard_top_left.append({
                "x": curr_width,
                "y": curr_height,
                "max_x": curr_width + shard_width,
                "max_y": curr_height + shard_height
            })
            curr_height = math.floor(shard_height + curr_height)
            
        if curr_height < vtt["imageHeight"]:
            shard_top_left.append({
                "x": curr_width,
                "y": curr_height,
                "max_x": vtt["imageHeight"],
                "max_y": vtt["imageWidth"]
            })
        curr_width = math.floor(curr_width + shard_width)
        if (curr_width + shard_width) > vtt["imageWidth"]:
            shard_width = math.floor(vtt["imageWidth"] - curr_width)
        curr_height = 0
            
    return shard_top_left

In [26]:
def create_shard(local, vtt, top_lefts):
    if len(top_lefts) <= 1:
        print(local)
        return
    
    from PIL import Image
    import math
    
    img = Image.open(local)
    display(img)
    
    for e in top_lefts:
        cp = img.copy()
        x1 = math.floor(e["x"])
        x2 = math.floor(e["max_x"])
        y1 = math.floor(e["y"])
        y2 = math.floor(e["max_y"])
        print(f"{x1},{y1} {x2},{y2}")
        img_shard = cp.crop((x1, y1, x2, y2))
        #img_shard = cp.resize(size=(x2 - x1, y2 - y1), box=(x1, y1, x2 - x1, y2 - y1))
        display(img_shard)
        

In [ ]:
for _, v in enumerate(metadata):
    vtt = v["vtt"]
    local= v["local"]
    cols = vtt["cols"]
    rows = vtt["rows"]
    top_lefts = create_image_shard_coords(vtt)
    create_shard(local, vtt, top_lefts)

In [48]:
metadata = (m for m in metadata if m["local"].find("illmarsh") == -1) 
metadata = list(metadata)
test_img = metadata[0]
print(test_img)

{'source': 'ScrapedData', 'gcsURI': "gs://fantasy-maps/ScrapedData/here's_a_little_mine_for_you._.40x40.jpg", 'computedBBoxes': [{'yMin': 0.0240234375, 'displayName': 'cell', 'yMax': 0.0509765625, 'xMin': 0.0240234375, 'xMax': 0.0509765625}, {'xMax': 0.0509765625, 'xMin': 0.0240234375, 'yMin': 0.0490234375, 'yMax': 0.07597656250000001, 'displayName': 'cell'}, {'yMax': 0.1009765625, 'xMax': 0.0509765625, 'xMin': 0.0240234375, 'displayName': 'cell', 'yMin': 0.07402343750000001}, {'yMax': 0.1259765625, 'displayName': 'cell', 'xMin': 0.0240234375, 'xMax': 0.0509765625, 'yMin': 0.0990234375}, {'xMin': 0.0240234375, 'xMax': 0.0509765625, 'yMax': 0.1509765625, 'displayName': 'cell', 'yMin': 0.1240234375}, {'yMin': 0.14902343750000002, 'yMax': 0.17597656250000002, 'xMax': 0.0509765625, 'xMin': 0.0240234375, 'displayName': 'cell'}, {'yMax': 0.2009765625, 'xMax': 0.0509765625, 'yMin': 0.17402343750000002, 'displayName': 'cell', 'xMin': 0.0240234375}, {'yMin': 0.1990234375, 'displayName': 'cell',

In [111]:
from imgaug import augmenters as iaa
help(iaa.Crop)

Help on class Crop in module imgaug.augmenters.size:

class Crop(CropAndPad)
 |  Crop(px=None, percent=None, keep_size=True, sample_independently=True, seed=None, name=None, random_state='deprecated', deterministic='deprecated')
 |  
 |  Crop images, i.e. remove columns/rows of pixels at the sides of images.
 |  
 |  This augmenter allows to extract smaller-sized subimages from given
 |  full-sized input images. The number of pixels to cut off may be defined
 |  in absolute values or as fractions of the image sizes.
 |  
 |  This augmenter will never crop images below a height or width of ``1``.
 |  
 |  **Supported dtypes**:
 |  
 |  See :class:`~imgaug.augmenters.size.CropAndPad`.
 |  
 |  Parameters
 |  ----------
 |  px : None or int or imgaug.parameters.StochasticParameter or tuple, optional
 |      The number of pixels to crop on each side of the image.
 |      Expected value range is ``[0, inf)``.
 |      Either this or the parameter `percent` may be set, not both at the same
 |